<h1> QRT Grand Data Challenge

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import IsolationForest
from category_encoders import TargetEncoder

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow as tf
tf.random.set_seed(42)

print("✅ Imports done")

✅ Imports done


In [19]:
X_train = pd.read_csv("data/X_train.csv")
X_test = pd.read_csv("data/X_test.csv")
y_train = pd.read_csv("data/y_train.csv")

print(X_train.shape, X_test.shape)
display(X_train.head())


(180245, 44) (7735, 44)


,ROW_ID,TS,ALLOCATION,RET_20,RET_19,RET_18,RET_17,RET_16,RET_15,RET_14,...,SIGNED_VOLUME_9,SIGNED_VOLUME_8,SIGNED_VOLUME_7,SIGNED_VOLUME_6,SIGNED_VOLUME_5,SIGNED_VOLUME_4,SIGNED_VOLUME_3,SIGNED_VOLUME_2,SIGNED_VOLUME_1,AVG_DAILY_TURNOVER
0,0,DATE_0001,ALLOCATION_01,-0.002477,0.004826,0.005374,-0.001688,-0.000152,-0.000685,-0.002217,...,-1.016154,-1.011450,-1.171714,-0.729594,-1.208138,-1.215123,-0.848346,-0.642461,-0.203447,0.054324
1,1,DATE_0001,ALLOCATION_02,0.006863,-0.005265,-0.004249,0.002686,-0.002638,0.003056,0.002712,...,0.896098,1.429419,0.946527,1.059767,0.988289,0.956915,0.943508,0.124168,0.081083,0.015669
2,2,DATE_0001,ALLOCATION_03,-0.005535,0.008541,0.005360,-0.002491,0.004679,-0.000848,-0.007197,...,-0.889142,-0.939257,-0.980370,-0.863196,-0.839662,-0.882459,-1.172723,-0.863937,-0.695998,0.057961
3,3,DATE_0001,ALLOCATION_04,0.003178,-0.001352,-0.004051,-0.001841,-0.005659,0.000627,0.006686,...,-1.788263,-0.807971,-1.587942,-0.042083,-1.356051,-1.007006,-1.821786,-0.455660,-1.090989,0.096004
4,4,DATE_0001,ALLOCATION_05,0.003359,-0.003349,-0.005460,0.000416,-0.003533,0.000913,0.005088,...,0.326148,1.013100,0.362135,0.774670,0.370484,-0.132558,-0.417645,-1.284208,-1.382900,0.005816


In [20]:
RET_features = [f'RET_{i}' for i in range(1, 20)]
SIGNED_VOLUME_features = [f'SIGNED_VOLUME_{i}' for i in range(1, 20)]
TURNOVER_features = ['AVG_DAILY_TURNOVER']


In [21]:
imputer = SimpleImputer(strategy="mean")
X_test[RET_features
    + SIGNED_VOLUME_features
    + TURNOVER_features] = imputer.fit_transform(X_test[RET_features
    + SIGNED_VOLUME_features
    + TURNOVER_features])

In [22]:
def feature_engineering(X):
    RET_features = [f'RET_{i}' for i in range(1, 21)]
    VOL_features = [f'SIGNED_VOLUME_{i}' for i in range(1, 21)]
    TURNOVER_features = ['AVG_DAILY_TURNOVER']

    for i in range(2, 21):
        X[f'RET_MEAN_{i}'] = X[RET_features[:i]].mean(axis=1)
        X[f'RET_STD_{i}'] = X[RET_features[:i]].std(axis=1)
        X[f'VOL_MEAN_{i}'] = X[VOL_features[:i]].mean(axis=1)
        X[f'VOL_STD_{i}'] = X[VOL_features[:i]].std(axis=1)

    # Trends
    X["RET_TREND_5"] = X["RET_20"] - X["RET_15"]
    X["VOL_TREND_5"] = X["SIGNED_VOLUME_20"] - X["SIGNED_VOLUME_15"]

    # Temporal drift encoding
    X["TS_num"] = X["TS"].str.extract("(\d+)").astype(int)
    X["VOL_PERF_RATIO"] = X[VOL_features].mean(axis=1) / (X[RET_features].std(axis=1) + 1e-9)

    return X, RET_features, VOL_features, TURNOVER_features

X_train, RET_features, VOL_features, TURNOVER_features = feature_engineering(X_train)
X_test, _, _, _ = feature_engineering(X_test)


<>:17: SyntaxWarning: invalid escape sequence '\d'
<>:17: SyntaxWarning: invalid escape sequence '\d'
/var/folders/tp/p2fdv61s7979qs0rlp6mh9qw0000gn/T/ipykernel_76339/518491733.py:17: SyntaxWarning: invalid escape sequence '\d'
  X["TS_num"] = X["TS"].str.extract("(\d+)").astype(int)
/var/folders/tp/p2fdv61s7979qs0rlp6mh9qw0000gn/T/ipykernel_76339/518491733.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[f'RET_STD_{i}'] = X[RET_features[:i]].std(axis=1)
/var/folders/tp/p2fdv61s7979qs0rlp6mh9qw0000gn/T/ipykernel_76339/518491733.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To ge

In [23]:
def oof_target_encoding(X, y, col, n_splits=5):
    X = X.copy().reset_index(drop=True)
    y = y.reset_index(drop=True)
    X[f"{col}_enc"] = np.nan
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    for train_idx, val_idx in kf.split(X):
        enc = TargetEncoder(smoothing=0.3)
        enc.fit(X.iloc[train_idx][[col]], (y.iloc[train_idx] > 0).astype(int))
        X.loc[val_idx, f"{col}_enc"] = enc.transform(X.iloc[val_idx][[col]]).values.flatten()

    enc.fit(X[[col]], (y > 0).astype(int))
    return X, enc

X_train, alloc_encoder = oof_target_encoding(X_train, y_train["target"], "ALLOCATION")
X_test["ALLOCATION_enc"] = alloc_encoder.transform(X_test[["ALLOCATION"]])


/var/folders/tp/p2fdv61s7979qs0rlp6mh9qw0000gn/T/ipykernel_76339/991618047.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test["ALLOCATION_enc"] = alloc_encoder.transform(X_test[["ALLOCATION"]])


In [24]:
scaler_ret = StandardScaler()
scaler_vol = StandardScaler()

X_train_ret = scaler_ret.fit_transform(X_train[RET_features])
X_test_ret = scaler_ret.transform(X_test[RET_features])

X_train_vol = scaler_vol.fit_transform(X_train[VOL_features])
X_test_vol = scaler_vol.transform(X_test[VOL_features])


In [25]:
def build_autoencoder(input_dim, latent_dim=5):
    inp = Input(shape=(input_dim,))
    encoded = Dense(12, activation='relu')(inp)
    encoded = Dense(latent_dim, activation='relu')(encoded)
    decoded = Dense(12, activation='relu')(encoded)
    out = Dense(input_dim, activation='linear')(decoded)
    return Model(inp, out), Model(inp, encoded)

autoencoder_ret, encoder_ret = build_autoencoder(20, latent_dim=5)
autoencoder_ret.compile(optimizer='adam', loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

autoencoder_ret.fit(
    X_train_ret, X_train_ret,
    validation_split=0.1,
    epochs=50,
    batch_size=256,
    callbacks=[early_stop],
    verbose=0
)


In [26]:
X_train_ret_latent = encoder_ret.predict(X_train_ret)
X_test_ret_latent = encoder_ret.predict(X_test_ret)

ret_latent_cols = [f"RET_LATENT_{i}" for i in range(X_train_ret_latent.shape[1])]
X_train[ret_latent_cols] = X_train_ret_latent
X_test[ret_latent_cols] = X_test_ret_latent


5633/5633 ━━━━━━━━━━━━━━━━━━━━ 1s 171us/step
242/242 ━━━━━━━━━━━━━━━━━━━━ 0s 203us/step


/var/folders/tp/p2fdv61s7979qs0rlp6mh9qw0000gn/T/ipykernel_76339/3045747527.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test[ret_latent_cols] = X_test_ret_latent
/var/folders/tp/p2fdv61s7979qs0rlp6mh9qw0000gn/T/ipykernel_76339/3045747527.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test[ret_latent_cols] = X_test_ret_latent
/var/folders/tp/p2fdv61s7979qs0rlp6mh9qw0000gn/T/ipykernel_76339/3045747527.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` ma

In [27]:
autoencoder_vol, encoder_vol = build_autoencoder(20, latent_dim=5)
autoencoder_vol.compile(optimizer='adam', loss='mse')
autoencoder_vol.fit(
    X_train_vol, X_train_vol,
    validation_split=0.1,
    epochs=50,
    batch_size=256,
    callbacks=[early_stop],
    verbose=0
)

X_train_vol_latent = encoder_vol.predict(X_train_vol)
X_test_vol_latent = encoder_vol.predict(X_test_vol)

vol_latent_cols = [f"VOL_LATENT_{i}" for i in range(X_train_vol_latent.shape[1])]
X_train[vol_latent_cols] = X_train_vol_latent
X_test[vol_latent_cols] = X_test_vol_latent


5633/5633 ━━━━━━━━━━━━━━━━━━━━ 1s 170us/step
242/242 ━━━━━━━━━━━━━━━━━━━━ 0s 209us/step


/var/folders/tp/p2fdv61s7979qs0rlp6mh9qw0000gn/T/ipykernel_76339/1872457523.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test[vol_latent_cols] = X_test_vol_latent
/var/folders/tp/p2fdv61s7979qs0rlp6mh9qw0000gn/T/ipykernel_76339/1872457523.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test[vol_latent_cols] = X_test_vol_latent
/var/folders/tp/p2fdv61s7979qs0rlp6mh9qw0000gn/T/ipykernel_76339/1872457523.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert`

In [28]:
iso = IsolationForest(contamination=0.02, random_state=42)
mask = iso.fit_predict(X_train[ret_latent_cols + vol_latent_cols]) == 1
X_train_clean = X_train[mask]
y_train_clean = y_train.loc[mask]


In [29]:
features_final = ret_latent_cols + vol_latent_cols + ["ALLOCATION_enc", "AVG_DAILY_TURNOVER"]

scaler_final = StandardScaler()
X_train_scaled = scaler_final.fit_transform(X_train_clean[features_final])
X_test_scaled = scaler_final.transform(X_test[features_final])

ridge = RidgeClassifier(alpha=1.0)
ridge.fit(X_train_scaled, (y_train_clean["target"] > 0).astype(int))

y_pred_test = ridge.predict(X_test_scaled)


In [30]:
submission = pd.DataFrame({
    "ROW_ID": X_test["ROW_ID"],
    "target": y_pred_test
})
submission.to_csv("submission_autoencoder_ridge.csv", index=False)
print("✅ Fichier de soumission généré avec succès !")


✅ Fichier de soumission généré avec succès !
